In [37]:
import pandas as pd

selected_features_labeled  = pd.read_csv("selected_features.csv",sep='\t')

In [28]:
selected_features_labeled

,SEQN,BPQ020,DIQ010,DID040,DIQ220,DIQ190A,DIQ190B,DIQ230,DID260,DIQ280,...,DIQ300D,PAQ180,PADACTIV,PADDURAT,SLD010H,LBXAPB,LBDINSI,LBXGH,RIAGENDR,RIDAGEMN
0,31132.0,1.0,1.0,63.0,0.0,1.0,1,1.0,55,6.9,...,2.53,2.0,42.0,40.0,7.0,75.00,53.94,7.10,1.0,842
1,31132.0,1.0,1.0,63.0,0.0,1.0,1,1.0,55,6.9,...,2.53,2.0,19.0,240.0,7.0,75.00,53.94,7.10,1.0,842
2,31134.0,1.0,2.0,0.0,0.0,1.0,1,0.0,0,0.0,...,0.00,3.0,42.0,30.0,7.0,111.00,87.06,5.90,1.0,882
3,31139.0,1.0,2.0,0.0,0.0,1.0,1,0.0,0,0.0,...,0.00,2.0,10.0,15.0,6.0,92.33,66.89,5.31,2.0,217
4,31139.0,1.0,2.0,0.0,0.0,1.0,1,0.0,0,0.0,...,0.00,2.0,40.0,30.0,6.0,92.33,66.89,5.31,2.0,217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958,41462.0,2.0,2.0,0.0,0.0,2.0,2,0.0,0,0.0,...,0.00,3.0,42.0,30.0,8.0,92.00,24.36,5.10,1.0,678
4959,41462.0,2.0,2.0,0.0,0.0,2.0,2,0.0,0,0.0,...,0.00,3.0,20.0,120.0,8.0,92.00,24.36,5.10,1.0,678
4960,41462.0,2.0,2.0,0.0,0.0,2.0,2,0.0,0,0.0,...,0.00,3.0,29.0,10.0,8.0,92.00,24.36,5.10,1.0,678
4961,41465.0,2.0,2.0,0.0,0.0,1.0,1,0.0,0,0.0,...,0.00,1.0,42.0,30.0,6.0,84.00,342.54,5.60,2.0,222


In [ ]:
selected_features_labeled.value_counts()
#zero_value_cols = selected_features_labeled.columns[(selected_features_labeled == 0).all()]
#zero_only_cols = selected_features_labeled.columns[(selected_features_labeled.eq(0)).all()]
zero_counts = (selected_features_labeled == 0).sum()

print("Total missing values: ", selected_features_labeled.isnull().sum().sum())
print("Missing values per column: \n", selected_features_labeled.isnull().sum())
print("Columns with all zero values: ", zero_counts)




Total missing values:  0
Missing values per column: 
 SEQN        0
BPQ020      0
DIQ010      0
DID040      0
DIQ220      0
DIQ190A     0
DIQ190B     0
DIQ230      0
DID260      0
DIQ280      0
DIQ300S     0
DIQ300D     0
PAQ180      0
PADACTIV    0
PADDURAT    0
SLD010H     0
LBXAPB      0
LBDINSI     0
LBXGH       0
RIAGENDR    0
RIDAGEMN    0
dtype: int64
Columns with all zero values:  SEQN           0
BPQ020         0
DIQ010         0
DID040      4733
DIQ220      4914
DIQ190A        0
DIQ190B        0
DIQ230      4733
DID260      4765
DIQ280      4785
DIQ300S     4733
DIQ300D     4733
PAQ180         0
PADACTIV       0
PADDURAT       3
SLD010H        0
LBXAPB       349
LBDINSI      372
LBXGH        329
RIAGENDR       0
RIDAGEMN      32
dtype: int64


In [53]:
zero_counts = (selected_features_labeled == 0).sum()
print("Number of zeros in the selected features:", zero_counts)

#drop columns
selected_features_labeled = selected_features_labeled.drop(columns=[
    'DID040', 'DIQ220', 'DIQ230', 'DID260', 'DIQ280', 'DIQ300S', 'DIQ300D'
], errors='ignore')

#drop rows
#selected_features_labeled = selected_features_labeled.drop(index=['LBXAPB', 'LBDINSI', 'LBXGH', 'PADDURAT', 'RIDAGEMN'], errors='ignore')

cols_to_check = ['LBXAPB', 'LBDINSI', 'LBXGH', 'PADDURAT', 'RIDAGEMN']

# Drop rows where any of the specified columns contain a zero
selected_features_labeled = selected_features_labeled[(selected_features_labeled[cols_to_check] != 0).all(axis=1)]

#print("Remaining columns:", selected_features_labeled.columns.tolist().count())
#print("Remaining rows:", selected_features_labeled.index.tolist().count())




Number of zeros in the selected features: SEQN        0
BPQ020      0
DIQ010      0
DIQ190A     0
DIQ190B     0
PAQ180      0
PADACTIV    0
PADDURAT    0
SLD010H     0
LBXAPB      0
LBDINSI     0
LBXGH       0
RIAGENDR    0
RIDAGEMN    0
dtype: int64


In [51]:
selected_features_labeled

## save cleaned data to a CSV file
def save_cleaned_data(df, data_path="selected_features_final.csv"):
    if not df.empty:
        df.to_csv(data_path, sep='\t', index=False)
    else:
        print("No data to save.")

save_cleaned_data(selected_features_labeled, data_path="selected_features_final.csv")


In [55]:
selected_features_labeled

,SEQN,BPQ020,DIQ010,DIQ190A,DIQ190B,PAQ180,PADACTIV,PADDURAT,SLD010H,LBXAPB,LBDINSI,LBXGH,RIAGENDR,RIDAGEMN
0,31132.0,1.0,1.0,1.0,1,2,42.0,40.0,7.0,75,53.94,7.1,1.0,842
1,31132.0,1.0,1.0,1.0,1,2,19.0,240.0,7.0,75,53.94,7.1,1.0,842
2,31134.0,1.0,2.0,1.0,1,3,42.0,30.0,7.0,111,87.06,5.9,1.0,882
17,31148.0,2.0,2.0,2.0,2,2,23.0,60.0,7.0,47,62.76,5.0,2.0,194
18,31150.0,2.0,2.0,2.0,2,4,43.0,60.0,8.0,75,23.46,5.0,1.0,959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958,41462.0,2.0,2.0,2.0,2,3,42.0,30.0,8.0,92,24.36,5.1,1.0,678
4959,41462.0,2.0,2.0,2.0,2,3,20.0,120.0,8.0,92,24.36,5.1,1.0,678
4960,41462.0,2.0,2.0,2.0,2,3,29.0,10.0,8.0,92,24.36,5.1,1.0,678
4961,41465.0,2.0,2.0,1.0,1,1,42.0,30.0,6.0,84,342.54,5.6,2.0,222


In [10]:
import pandas as pd
## preDM filter
def main():
    combined_selected_features = pd.read_csv('selected_features_final.csv', sep='\t')
    preDM_criteria = (combined_selected_features['LBDINSI'] >= 100) | (combined_selected_features['LBXAPB'] >= 110) | (combined_selected_features['LBXGH'] >= 5.5)

    preDMCondFilter(combined_selected_features, preDM_criteria)
    print('test line')

def preDMCondFilter(combined_selected_features, preDM_criteria):
    
    #combined_selected_features.columns = combined_selected_features.columns.str.strip()

    combined_selected_features['PreDM'] = preDM_criteria.astype(int)

    combined_selected_features.to_csv('selected_features_labeled_final.csv', sep='\t', index=False)
    
if __name__ == "__main__":
    main()

test line


NameError: name 'selected_features_labeled' is not defined

In [ ]:


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepareData():
    labeled_selected_features = pd.read_csv('../selected_features_labeled.csv', sep='\t', header=0)

    X_train, X_test, y_train, y_test = encode_and_split(labeled_selected_features)

    X_train_scaled, X_test_scaled = scaleFeatures(X_train, X_test)
 
    #X_resampled, y_resampled = handle_class_imbalance(X_train_scaled, y_train)

    return X_train, X_test, X_test_scaled, y_test

def encode_and_split(labeled_selected_features):

    #X = labeled_selected_features[['BPQ020', 'PADDURAT', 'SLD010H', 'RIAGENDR', 'LBXAPB', 'LBDINSI', 'LBXGH']]
    X = labeled_selected_features[['BPQ020', 'PADDURAT',  'SLD010H']]
    Y = labeled_selected_features["PreDM"].copy()
   
    X = X.loc[Y.index]

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, train_size=0.7, random_state=42)

    return X_train, X_test, y_train, y_test

def scaleFeatures(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

# oversample the minority class using SMOTE
# avoid overfitting
# improves performance on imbalanced datasets


test


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

def main():
    X_train, X_test, X_test_scaled, y_test = prepareData()
    accuracy = train_evaluate_model(X_train, X_test, X_test_scaled, y_test)
    return accuracy

def train_evaluate_model(X_train, X_test, X_test_scaled, y_test):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, X_test)

    y_pred = dt_model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Decision Tree Accuracy: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))

    return accuracy

if __name__ == "__main__":
    main()

ValueError: Input y contains NaN.

In [15]:
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [18]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 2.8 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 2.8 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.1 MB 2.9 MB/s eta 0:00:04
   ---------- ----------------------------- 2.9/11.1 MB 2.9 MB/s eta 0:00:03
   ------------ --------------------------- 3.4/11.1 MB 2.8 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.1 MB 2.8 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.1 MB 2.8 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.1 MB 2.8 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.1 MB 2.9 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.1 MB 2.9 MB/s eta 0:00:02
   ------------------------